In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.optimize as opt
from sklearn.model_selection import train_test_split
import random as rnd

In [55]:
data = pd.read_excel("C:/Users/Niku/Documents/Semester8/DataFiles/datasetnorm1.xlsx")
data.shape

(1994, 121)

In [56]:
def kl(x1, x2):
    temp =  np.dot(x1, x2.T)
    return temp

In [57]:
def kq(x1, x2):
    temp = np.dot(x1, x2.T)
    temp = temp ** 2
    return temp

In [58]:
def test(X):
    temp = np.dot(w.T, X.T)
    return np.sign(temp + b)

In [59]:
def findb(X, y, w):
    temp = y - np.dot(w.T, X.T)
    return np.mean(temp)

In [60]:
def findw(alpha, y, X):
    return np.dot(alpha * y, X)

In [61]:
def findL(a1, a2, C, y1, y2):
    if y1 == y2:
        L = max(0, a2 + a1 - C)
    else:
        L = max(0, a2 - a1)
    return L

In [62]:
def findH(a1, a2, C, y1, y2):
    if y1 == y2:
        H = min(C, a2 + a1)
    else:
        H = min(C, C + a2 - a1)
    return H

In [63]:
max_iter=10000
kernel ='quadratic'
C = 1.0
epsilon=0.001

In [64]:
def train(X, y, epsilon, C, max_iter, kernel):
    m, n = X.shape[0], X.shape[1]
    alpha = np.zeros((m))
    iterations = 0
    while iterations < max_iter:
        iterations += 1
        alpha_prev = np.copy(alpha)
        for i in range(0, m):
            j = i
            while j == i:
                j = rnd.randint(0,m-1)
            x1 = X[i, :]
            x2 = X[j, :]
            y1 = y[i]
            y2 = y[j]
            a1 = alpha[i]
            a2 = alpha[j]
            
            if kernel == 'quadratic':
                k = kq(x1, x1) + kq(x2, x2) - 2 * kq(x1, x2)
            else:
                k = kl(x1, x1) + kl(x2, x2) - 2 * kl(x1, x2)
            
            w = np.dot(alpha * y, X)
            temp = y - np.dot(w.T, X.T)
            b = np.mean(temp)
            
            temp1 = np.dot(w.T, x1.T)
            temp1 = temp1 + b
            E1 = np.sign(temp1) - y1
            
            temp2 = np.dot(w.T, x2.T)
            temp2 = temp2 + b
            E2 = np.sign(temp2) - y2
            
            L = findL(a1, a2, C, y1, y2)
            H = findH(a1, a2, C, y1, y2)
            
            alpha[j] = a2 + float(y2 * (E1 - E2))/k
            
            if alpha[j] >= H:
                alpha[j] = H
            if alpha[j] <= L:
                alpha[j] = L

            alpha[i] = a1 + y1*y2 * (a2 - alpha[j])
            
            alphadiff = alpha - alpha_prev
            diff = np.linalg.norm(alphadiff)
            if diff < epsilon:
                break

    return b, w

In [65]:
rows = data.shape[0]
cols = data.shape[1]
y = data.iloc[:, cols-1]
X = data.iloc[:, 1:cols-1]
X = np.array(X)
y = np.array(y)
y[y == 0] = -1

In [71]:
totalacc = 0.0
max_iter=[1000, 10000]
kernel ='quadratic'
C = [0.1, 1.0]
epsilon=[0.01, 0.001]
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn import metrics
count = 0
prec = np.zeros((8,2))
recal = np.zeros((8,2))
f1 = np.zeros((8,2))
acc = np.zeros((8,2))
auc =  np.zeros((8,2))
for i in range(2):
    for j in range(2):
        for k in range(2):
            e = epsilon[i]
            C_ = C[j]
            miter = max_iter[k]
            totalacc = 0
            for l in range(2):
                print e, C_, miter
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5)
                X_train.shape, y_train.shape
                [b, w] = train(X_train, y_train, e, C_, miter, kernel)
                y_pred = test(X_test)
                from sklearn.metrics import classification_report
                #print(classification_report(y_test, y_pred))
                prec[count, l] = precision_score(y_test, y_pred, average="weighted")
                recal[count, l] = recall_score(y_test, y_pred, average="weighted")
                f1[count, l] = f1_score(y_test, y_pred, average="weighted")
                acc[count, l] = accuracy_score(y_test, y_pred)
                fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred) 
                auc[count, l] = metrics.auc(fpr, tpr)
                #print 'accuracy = {0}%'.format(accuracy * 100)
            print (prec[count, 0]+prec[count,1])*1.0/2, (recal[count, 0]+recal[count,1])*1.0/2, (f1[count, 0]+f1[count,1])*1.0/2, (acc[count, 0]+acc[count,1])*1.0/2,(auc[count, 0]+auc[count,1])*1.0/2 
            count += 1     

0.01 0.1 1000
0.01 0.1 1000
0.8729629321 0.873119358074 0.872440685267 0.873119358074 0.864178628699
0.01 0.1 10000
0.01 0.1 10000
0.873614246236 0.873620862588 0.873090602366 0.873620862588 0.865614589805
0.01 1.0 1000
0.01 1.0 1000
0.916833611773 0.916248746239 0.915756480249 0.916248746239 0.908840925721
0.01 1.0 10000
0.01 1.0 10000
0.9186512215 0.916750250752 0.91596874205 0.916750250752 0.906987859331
0.001 0.1 1000
0.001 0.1 1000
0.875509520701 0.875626880642 0.875341661441 0.875626880642 0.870292636642
0.001 0.1 10000
0.001 0.1 10000
0.873120785912 0.873119358074 0.8724621571 0.873119358074 0.862680565172
0.001 1.0 1000
0.001 1.0 1000
0.92482020339 0.923269809428 0.922670141677 0.923269809428 0.914909398435
0.001 1.0 10000
0.001 1.0 10000
0.91660586398 0.915245737212 0.914584363288 0.915245737212 0.906423740768


In [68]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

         -1       0.92      0.95      0.94       598
          1       0.93      0.88      0.90       399

avg / total       0.92      0.92      0.92       997

